In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing import sequence

## Load data

Modify `imdb.py` in keras library

```
-  with np.load(path) as f:
+  with np.load(path, allow_pickle=True) as f:
```

Constraining the dataset to the top 5,000 words. Split the dataset into train (50%) and test (50%) sets.

X_train, X_test 25,000 encoded word sequence lists.

See [Keras dataset doc](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification)

In [17]:
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

# X_train ndarray(25000,) of lists

Truncate and pad the input sequences so that they are all the same length for modeling. The model will learn the zero values carry no information so indeed the sequences are not the same length in terms of content, but same length vectors is required to perform the computation in Keras.

In [18]:
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

# X_train ndarray(25000,500)

SAVE_DATASET = False

if SAVE_DATASET:
    np.save('x_train.npy', X_train)
    np.save('x_test.npy', X_test)
    np.save('y_train.npy', y_train)
    np.save('y_test.npy', y_test)

## Model

### Embedding layer

Keras provides a convenient way to convert positive integer representations of words into a word embedding by an Embedding layer.

According to Keras [doc](https://keras.io/layers/embeddings/)

Embedding layer turns positive integers (indexes) into dense vectors of fixed size. eg. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]

This layer can only be used as the first layer in a model.

### LSTM and Dense layer

According to Keras [doc]()

LSTM layer

units: Positive integer, dimensionality of the output space.

return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence. Default: False

The output is the hidden state, and it contains information on previous inputs.

Dense layer

Transform the hidden state to prediction.

In [4]:
embedding_dim = 32
hidden_dim = 100

model = Sequential()
model.add(Embedding(top_words, embedding_dim, input_length=max_review_length))
model.add(LSTM(hidden_dim))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 17s 692us/sample - loss: 0.4906 - accuracy: 0.7595 - val_loss: 0.3265 - val_accuracy: 0.8654
Epoch 2/3
25000/25000 [==============================] - 14s 576us/sample - loss: 0.3047 - accuracy: 0.8762 - val_loss: 0.3337 - val_accuracy: 0.8700
Epoch 3/3
25000/25000 [==============================] - 14s 577us/sample - loss: 0.4280 - accuracy: 0.8255 - val_loss: 0.4275 - val_accuracy: 0.8156


## Evaluation

In [6]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 81.56%


## References

1. [Machine Learning Mastery: Sequence Classification with LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/)

2. [Machine Learning Mastery: The 5 Step Life-Cycle for Long Short-Term Memory Models in Keras](https://machinelearningmastery.com/5-step-life-cycle-long-short-term-memory-models-keras/) General LSTM architecture.

3. [Medium: Illustrated Guide to LSTM’s and GRU’s: A step by step explanation](https://medium.com/coinmonks/word-level-lstm-text-generator-creating-automatic-song-lyrics-with-neural-networks-b8a1617104fb) Learn how LSTM works.